### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Arabic
Lang Id = 9
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ماذا عن هنا,426,429,ماذا عن هنا,5fFRsiBpyWE,https://www.youtube.com/watch?v=5fFRsiBpyWE&t=...
1,و ماذا ما,487,489,و ماذا ما,kI4ILlAsHtg,https://www.youtube.com/watch?v=kI4ILlAsHtg&t=...
2,هل هذا هو,102,104,هل هذا هو,FLW2e3o5q5w,https://www.youtube.com/watch?v=FLW2e3o5q5w&t=...
3,أنا و أنت,702,704,أنا و أنت,ftx8qb8WYqY,https://www.youtube.com/watch?v=ftx8qb8WYqY&t=...
4,ما كل هذا,560,561,ما كل هذا,8mAA3d5GLSU,https://www.youtube.com/watch?v=8mAA3d5GLSU&t=...
5,كان هناك,359,363,كان هناك,l8o7hOEG29E,https://www.youtube.com/watch?v=l8o7hOEG29E&t=...
6,و كل هذا,64,67,و كل هذا,EGvt09bhR8A,https://www.youtube.com/watch?v=EGvt09bhR8A&t=64s
7,ان هناك,3333,3335,ان هناك,T9SDwMP9SsE,https://www.youtube.com/watch?v=T9SDwMP9SsE&t=...
8,نعم أنت,157,158,نعم أنت,FLW2e3o5q5w,https://www.youtube.com/watch?v=FLW2e3o5q5w&t=...
9,ليس لا,155,157,ليس لا,svmVViL00wQ,https://www.youtube.com/watch?v=svmVViL00wQ&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,ولكن ماذا تريد أن تعرف من أجل,4903,4905,ولكن ماذا تريد أن تعرف من أجل,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
1,لا يمكنك فعل الكثير ولكن فقط,15058,15060,لا يمكنك فعل الكثير ولكن فقط انتظر و,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
2,لن تعرف حتى ما إذا كان هناك,8892,8894,والتي لن تعرف حتى ما إذا كان هناك ملف,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
3,أن يكون العمل اليوم أفضل من,481,484,أن يكون العمل اليوم أفضل من الأمس,WfHZLLJ3iCk,https://www.youtube.com/watch?v=WfHZLLJ3iCk&t=...
4,ولكن كما قلت من قبل في هذا,1864,1867,ولكن كما قلت من قبل في هذا الكتاب لك,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
5,ربما لا شيء سوف يكون أكثر,9992,9995,ربما لا شيء سوف يكون أكثر احتمالا,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
6,ولكن ماذا يحدث عندما يكون,5494,5497,ولكن ماذا يحدث عندما يكون الخطأ,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
7,لا تعرف في ذلك الوقت كيف,3735,3738,لا تعرف في ذلك الوقت كيف تحدد,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
8,لقد قلت بالتأكيد هو من,462,464,لقد قلت بالتأكيد هو من اوروبا,GkXHG8WxMAQ,https://www.youtube.com/watch?v=GkXHG8WxMAQ&t=...
9,في ذلك الوقت لم يكن قد,313,316,في ذلك الوقت لم يكن قد تقدم,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,و هنا أظن أنني قلت لكم مرحبا بكم جميعا مرحبا ب...,130,133,و هنا أظن أنني قلت لكم مرحبا بكم جميعا مرحبا ب...,mQpfOCan7F4,https://www.youtube.com/watch?v=mQpfOCan7F4&t=...
1,ربما تستطيع انت ان تعمل بطريقة افضل من الطريقة...,776,782,ربما تستطيع انت ان تعمل بطريقة افضل من الطريقة...,vyu7m7rp4aE,https://www.youtube.com/watch?v=vyu7m7rp4aE&t=...
2,لكن ليس كل رجل أفضل من المرأة بل قد يوجد من ال...,176,181,لكن ليس كل رجل أفضل من المرأة بل قد يوجد من ال...,TtCm6wfb1n4,https://www.youtube.com/watch?v=TtCm6wfb1n4&t=...
3,تذكر بأنه ليس عليك أن تعرف العائلة على أنها مج...,173,179,تذكر بأنه ليس عليك أن تعرف العائلة على أنها مج...,zWYpN9XepFo,https://www.youtube.com/watch?v=zWYpN9XepFo&t=...
4,هذه السيارة فهو يعرف الطريق كما أنه ذاهب إلى هناك,663,669,اتبع هذه السيارة فهو يعرف الطريق كما أنه ذاهب ...,4ConT24MpmE,https://www.youtube.com/watch?v=4ConT24MpmE&t=...
...,...,...,...,...,...,...
195,حسنا كان عندي هذا النوع من,101,104,حسنا كان عندي هذا النوع من القماش,0X8LKM3nXbQ,https://www.youtube.com/watch?v=0X8LKM3nXbQ&t=...
196,ثم يجب أن تعرف ذلك بالنسبة,9203,9205,ثم يجب أن تعرف ذلك بالنسبة لأوروبا,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
197,بشكل عام هو الأول لأنه فقط,9099,9102,بشكل عام هو الأول لأنه فقط,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
198,ما أريد أن أخبرك به هو أن,11462,11464,ما أريد أن أخبرك به هو أن الدراسة,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ومع ذلك أخبرني أن أذهب إلى هناك بمفردي لأنه لم...,1350,1358,ومع ذلك أخبرني أن أذهب إلى هناك بمفردي لأنه لم...,ENsy8s3XBMU,https://www.youtube.com/watch?v=ENsy8s3XBMU&t=...
1,في كثير من الأحيان يحدث لنا موقف مزعج في صورة ...,0,7,في كثير من الأحيان يحدث لنا موقف مزعج في صورة ...,Ibz7Oe_MW84,https://www.youtube.com/watch?v=Ibz7Oe_MW84&t=0s
2,فى هذا الفيديو سوف أتحدث عن واحد من أهم الأشيا...,2,7,فى هذا الفيديو سوف أتحدث عن واحد من أهم الأشيا...,JHJSM2gehd4,https://www.youtube.com/watch?v=JHJSM2gehd4&t=2s
3,هي التي تفعل كل تلك الأشياء التي لا يريد فعلها...,156,164,هي التي تفعل كل تلك الأشياء التي لا يريد فعلها...,Fm213BhZgsU,https://www.youtube.com/watch?v=Fm213BhZgsU&t=...
4,و لكن عندما يتعلق الأمر بالحديث فهو غير متأكد ...,85,94,و لكن عندما يتعلق الأمر بالحديث فهو غير متأكد ...,Fm213BhZgsU,https://www.youtube.com/watch?v=Fm213BhZgsU&t=85s
...,...,...,...,...,...,...
195,تبحث الفتيات عن أكثر الأحلام لديها وهو تفسير حلم,5,11,تبحث الفتيات عن أكثر الأحلام لديها وهو تفسير ح...,1Anb66K0oes,https://www.youtube.com/watch?v=1Anb66K0oes&t=5s
196,القواعد العسكرية وليست موقع الذي يخرج منه الجنود,775,781,القواعد العسكرية وليست موقع الذي يخرج منه الجن...,db0QDwsxMqs,https://www.youtube.com/watch?v=db0QDwsxMqs&t=...
197,على الرغم من أنه يمكن أن يكون مشكلة حقيقية عندما,15397,15400,على الرغم من أنه يمكن أن يكون مشكلة حقيقية عندما,k9BFEbTzIHw,https://www.youtube.com/watch?v=k9BFEbTzIHw&t=...
198,كان يجلس بها الجنود أما البرج الرئيسي هو أهم جزء,92,95,في غرف كان يجلس بها الجنود أما البرج الرئيسي ه...,8itG2MX3KjI,https://www.youtube.com/watch?v=8itG2MX3KjI&t=92s


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,إنها شخص يستخدم عقله و لا يستمع حقا لما يقوله ...,178,188,إنها شخص يستخدم عقله و لا يستمع حقا لما يقوله ...,TwJ2E6BHwkE,https://www.youtube.com/watch?v=TwJ2E6BHwkE&t=...
1,إنها تعتقد أنه من الأفضل إذا لم تظهر ما تشعر ب...,186,196,إنها تعتقد أنه من الأفضل إذا لم تظهر ما تشعر ب...,Fm213BhZgsU,https://www.youtube.com/watch?v=Fm213BhZgsU&t=...
2,فى هذا الفيديو سوف أتحدث عن واحد من أهم الأشيا...,2,8,فى هذا الفيديو سوف أتحدث عن واحد من أهم الأشيا...,JHJSM2gehd4,https://www.youtube.com/watch?v=JHJSM2gehd4&t=2s
3,هي التي تفعل كل تلك الأشياء التي لا يريد فعلها...,156,165,هي التي تفعل كل تلك الأشياء التي لا يريد فعلها...,Fm213BhZgsU,https://www.youtube.com/watch?v=Fm213BhZgsU&t=...
4,ومع ذلك أخبرني أن أذهب إلى هناك بمفردي لأنه لم...,1350,1358,ومع ذلك أخبرني أن أذهب إلى هناك بمفردي لأنه لم...,ENsy8s3XBMU,https://www.youtube.com/watch?v=ENsy8s3XBMU&t=...
...,...,...,...,...,...,...
195,اخر مرة لقد تحدثت عن كونها رياضة وأنا لم اكن ا...,461,465,اخر مرة لقد تحدثت عن كونها رياضة وأنا لم اكن ا...,po0G1Q8aOQw,https://www.youtube.com/watch?v=po0G1Q8aOQw&t=...
196,سؤال هل تبقى الشركة في السوق الروسية الرد هناك...,872,880,سؤال هل تبقى الشركة في السوق الروسية الرد هناك...,b0KoulARFzE,https://www.youtube.com/watch?v=b0KoulARFzE&t=...
197,أن الزوجة لا يوجد عندها الفيروس فقط الزوج الذي...,3438,3447,نفترض أن الزوجة لا يوجد عندها الفيروس فقط الزو...,yr2iKTA_gHM,https://www.youtube.com/watch?v=yr2iKTA_gHM&t=...
198,و تؤثر هذه السموم على الكبد نحن نعلم أن من وظا...,544,551,و تؤثر هذه السموم على الكبد نحن نعلم أن من وظا...,H9q7Mfu6aR0,https://www.youtube.com/watch?v=H9q7Mfu6aR0&t=...


In [14]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,سؤال ماذا يمكننا أن نفعل لمساعدتك في تحقيق صيا...,761,773,سؤال ماذا يمكننا أن نفعل لمساعدتك في تحقيق صيا...,b0KoulARFzE,https://www.youtube.com/watch?v=b0KoulARFzE&t=...
1,إنها شخص يستخدم عقله و لا يستمع حقا لما يقوله ...,178,190,إنها شخص يستخدم عقله و لا يستمع حقا لما يقوله ...,TwJ2E6BHwkE,https://www.youtube.com/watch?v=TwJ2E6BHwkE&t=...
2,وهذا ينطبق أيضا على العمل لأنه بغض النظر عن مد...,468,480,وهذا ينطبق أيضا على العمل لأنه بغض النظر عن مد...,xpNHqW98nLw,https://www.youtube.com/watch?v=xpNHqW98nLw&t=...
3,وقال المتحدث باسم الوزارة نيد برايس إن الولايا...,334,348,وقال المتحدث باسم الوزارة نيد برايس إن الولايا...,M1xNQMW6B3M,https://www.youtube.com/watch?v=M1xNQMW6B3M&t=...
4,الحقيقة لا يتعلق الأمر بمرور الوقت بسرعة فنحن ...,357,368,الحقيقة لا يتعلق الأمر بمرور الوقت بسرعة فنحن ...,zWYpN9XepFo,https://www.youtube.com/watch?v=zWYpN9XepFo&t=...
...,...,...,...,...,...,...
195,يمكنك الاختيار من بينها حسب ما يناسبك سواء كان...,153,160,يمكنك الاختيار من بينها حسب ما يناسبك سواء كان...,94JEo3jV0z4,https://www.youtube.com/watch?v=94JEo3jV0z4&t=...
196,كل هذه الأسباب تؤدي إلى ضغط في البطن وبالتالي ...,283,292,كل هذه الأسباب تؤدي إلى ضغط في البطن وبالتالي ...,Hp5mXsWIVIo,https://www.youtube.com/watch?v=Hp5mXsWIVIo&t=...
197,كان هذا ما لدينا أصدقائي لهذا اليوم شكرا جزيلا...,163,167,كان هذا ما لدينا أصدقائي لهذا اليوم شكرا جزيلا...,j_bs6ZD2amg,https://www.youtube.com/watch?v=j_bs6ZD2amg&t=...
198,كان هذا ما لدينا أصدقائي لهذا اليوم شكرا جزيلا...,498,502,كان هذا ما لدينا أصدقائي لهذا اليوم شكرا جزيلا...,G1FIfZMgqHM,https://www.youtube.com/watch?v=G1FIfZMgqHM&t=...


In [15]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,كان لابد من القيام بهذا العمل من أجل حماية الك...,600,616,كان لابد من القيام بهذا العمل من أجل حماية الك...,O6xqdZyN4UM,https://www.youtube.com/watch?v=O6xqdZyN4UM&t=...
1,أنت مسؤول عن تجربتك مع الواقع لا تدع شخصا غير ...,431,450,أنت مسؤول عن تجربتك مع الواقع لا تدع شخصا غير ...,poI4dI1ljx4,https://www.youtube.com/watch?v=poI4dI1ljx4&t=...
2,في هذا الموقف هو أنه لا يمكنك مساعدة أولئك الذ...,338,355,المفارقة في هذا الموقف هو أنه لا يمكنك مساعدة ...,U358NsXAcgA,https://www.youtube.com/watch?v=U358NsXAcgA&t=...
3,حتي وإن عاد الموقع في وقت متأخر لا مشكلة المهم...,1549,1587,حتي وإن عاد الموقع في وقت متأخر لا مشكلة المهم...,b0KoulARFzE,https://www.youtube.com/watch?v=b0KoulARFzE&t=...
4,يتبع التوقيت العالمي وليس توقيت روسيا وبالتالي...,31,60,أولا موقع اي ماركتنج يتبع التوقيت العالمي وليس...,jrOGR5KM_G4,https://www.youtube.com/watch?v=jrOGR5KM_G4&t=31s
...,...,...,...,...,...,...
195,حسنا الأزمات هنا لتبقى ولن تنتهي أبدا إن أنت و...,72,81,حسنا الأزمات هنا لتبقى ولن تنتهي أبدا إن أنت و...,ugE2RlhZK3w,https://www.youtube.com/watch?v=ugE2RlhZK3w&t=72s
196,بغض النظر عن مقدار الديكور الذي تريده أو حجم غ...,59,67,بغض النظر عن مقدار الديكور الذي تريده أو حجم غ...,94JEo3jV0z4,https://www.youtube.com/watch?v=94JEo3jV0z4&t=59s
197,قد يكون هذا بسبب الموت أو بسبب الإنفصال عن حبي...,27,35,قد يكون هذا بسبب الموت أو بسبب الإنفصال عن حبي...,C7bCCraI99s,https://www.youtube.com/watch?v=C7bCCraI99s&t=27s
198,اما مثل هذا الذي يقول ولا يعرف وفي مؤتمر دولي ...,294,306,اما مثل هذا الذي يقول ولا يعرف وفي مؤتمر دولي ...,fFa0yGyd1kY,https://www.youtube.com/watch?v=fFa0yGyd1kY&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1671799298
  nanos: 982042000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass